# Sample Data Seeder

- 4 customers
- 4-5 restaurants per city (Mumbai, Bangalore)
- 5 successful payments per customer with corresponding orders

This notebook resets existing data for a clean demo run.

In [1]:
from sqlmodel import Session, select, delete
from App.database import init_db, engine
from App.models import (
    Customers, Payments, Orders, Restaurants,
    PaymentStatus, PaymentType, OrderFoodItem, RestaurantAreaName
)
import random
from datetime import datetime, timedelta, timezone

# Initialize tables if not present
init_db()

# Helper: reset existing data for repeatable demos
def reset_data():
    with Session(engine) as session:
        session.exec(delete(Orders))
        session.exec(delete(Payments))
        session.exec(delete(Restaurants))
        session.exec(delete(Customers))
        session.commit()

reset_data()
print("Database reset complete.")

Database reset complete.


In [2]:
# Seed restaurants: 4-5 in each city
mumbai_names = ["Bombay Bites", "Marine Spice", "Gateway Grills", "Colaba Curry", "Dadar Deli"]
blr_names = ["Garden Greens", "Indiranagar Eats", "Koramangala Kitchen", "MG Road Meals", "Whitefield Wraps"]

random.shuffle(mumbai_names)
random.shuffle(blr_names)
mumbai_pick = mumbai_names[:random.randint(4,5)]
blr_pick = blr_names[:random.randint(4,5)]

with Session(engine) as session:
    for name in mumbai_pick:
        session.add(Restaurants(name=name, area=RestaurantAreaName.MUMBAI))
    for name in blr_pick:
        session.add(Restaurants(name=name, area=RestaurantAreaName.BANGALORE))
    session.commit()

print(f"Created {len(mumbai_pick)} Mumbai restaurants and {len(blr_pick)} Bangalore restaurants.")

Created 4 Mumbai restaurants and 5 Bangalore restaurants.


In [3]:
# Seed customers (4)
customers_seed = [
    {"google_id": "gid_1", "name": "Alice Johnson", "age": 28},
    {"google_id": "gid_2", "name": "Bob Singh", "age": 34},
    {"google_id": "gid_3", "name": "Chetan Rao", "age": 26},
    {"google_id": "gid_4", "name": "Diya Patel", "age": 31},
]

with Session(engine) as session:
    for c in customers_seed:
        session.add(Customers(**c))
    session.commit()

print("Created 4 customers.")

Created 4 customers.


In [4]:
# Create 5 successful payments and corresponding orders for each customer
items = [
    OrderFoodItem.VEG_MANCHURIAN,
    OrderFoodItem.CHICKEN_MANCHURIAN,
    OrderFoodItem.VEG_FRIED_RICE,
    OrderFoodItem.CHICKEN_FRIED_RICE,
]

random.seed(42)
now = datetime.now(timezone.utc)

# Helpers to generate dates
first_of_this_month = datetime(now.year, now.month, 1, tzinfo=timezone.utc)
last_month_end = first_of_this_month - timedelta(seconds=1)
first_of_last_month = datetime(last_month_end.year, last_month_end.month, 1, tzinfo=timezone.utc)

def random_dt_in_last_month():
    span_days = (last_month_end.date() - first_of_last_month.date()).days
    offset = random.randint(0, max(span_days, 0))
    # randomize time within the day
    hour = random.randint(9, 22)
    minute = random.randint(0, 59)
    second = random.randint(0, 59)
    d = first_of_last_month + timedelta(days=offset, hours=hour, minutes=minute, seconds=second)
    return d


def random_dt_past_days(days:int=7):
    offset = random.randint(0, max(days-1, 0))
    hour = random.randint(9, 22)
    minute = random.randint(0, 59)
    second = random.randint(0, 59)
    return now - timedelta(days=offset, hours=(23-hour), minutes=(59-minute), seconds=(59-second))

with Session(engine) as session:
    restaurants = session.exec(select(Restaurants)).all()
    customers = session.exec(select(Customers)).all()

    mumbai_restaurants = [r for r in restaurants if r.area == RestaurantAreaName.MUMBAI]
    blr_restaurants = [r for r in restaurants if r.area == RestaurantAreaName.BANGALORE]

    for cust in customers:
        for i in range(5):
            # Amount and type
            amt = round(random.uniform(150, 800), 2)
            ptype = random.choice([PaymentType.CARD, PaymentType.UPI])

            # Decide scenario and timestamps
            if i == 2 and mumbai_restaurants:
                # Ensure at least one Mumbai order last month
                created_at_dt = random_dt_in_last_month()
                restaurant = random.choice(mumbai_restaurants)
                item = random.choice(items)
            elif i in (0, 1) and blr_restaurants:
                # Ensure at least two veg orders in Bangalore (for veg earnings)
                created_at_dt = random_dt_past_days(7)
                restaurant = random.choice(blr_restaurants)
                item = random.choice([OrderFoodItem.VEG_MANCHURIAN, OrderFoodItem.VEG_FRIED_RICE])
            else:
                # Spread across past 7 days and random city
                created_at_dt = random_dt_past_days(7)
                # random city
                if blr_restaurants and (not mumbai_restaurants or random.random() < 0.5):
                    restaurant = random.choice(blr_restaurants)
                else:
                    restaurant = random.choice(mumbai_restaurants)
                item = random.choice(items)

            # Create payment and order with aligned timestamps
            payment = Payments(
                status=PaymentStatus.PASS,
                payment_type=ptype,
                amount=amt,
                currency="INR",
                customer_id=cust.id,
                created_at=created_at_dt,
            )
            session.add(payment)
            session.commit()
            session.refresh(payment)

            order = Orders(
                item_name=item,
                transaction_id=payment.transaction_id,
                restaurant_id=restaurant.restaurant_id,
                customer_id=cust.id,
                created_at=created_at_dt,
            )
            session.add(order)
            session.commit()

print("Created payments and orders: 5 per customer with realistic timestamps and locations.")

Created payments and orders: 5 per customer with realistic timestamps and locations.


In [5]:
# Quick summary
with Session(engine) as session:
    total_customers = session.exec(select(Customers)).all()
    total_restaurants = session.exec(select(Restaurants)).all()
    total_payments = session.exec(select(Payments)).all()
    total_orders = session.exec(select(Orders)).all()

print({
    'customers': len(total_customers),
    'restaurants': len(total_restaurants),
    'payments': len(total_payments),
    'orders': len(total_orders),
})

{'customers': 4, 'restaurants': 9, 'payments': 20, 'orders': 20}
